In [ ]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline

DATASET_PATH = "datasets/"

def load_song_data(dataset_path=DATASET_PATH):
    df = pd.read_csv(dataset_path + 'year-prediction-msd-train.txt', sep=",", header=None)
    return df

In [ ]:
#songs é um DataFrame
songs = load_song_data()

In [ ]:
songs_x = songs.iloc[:,1:20]
songs_y = songs.iloc[:,0]
#songs_x = songs_x.values.reshape(-1,1)
songs_y = songs_y.values.reshape(-1,1)

In [ ]:
#Normalizing
num_pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
])
songs_x_prepared = num_pipeline.fit_transform(songs_x)

In [ ]:
# Pegando os primeiros 20868 exemplos pra teste
songs_x_train, songs_x_test, songs_y_train, songs_y_test = train_test_split(songs_x_prepared, songs_y, test_size=0.045, random_state=0)

In [ ]:
# Train the model using the training sets
for count, degree in enumerate([2]):
    regr = make_pipeline(PolynomialFeatures(2), Ridge())
    regr.fit(songs_x_train, songs_y_train)
    songs_y_pred = regr.predict(songs_x_test)
    #plt.plot(songs_x_test, songs_y_pred, linewidth=1, label="degree %d" % 2)

In [ ]:
# The coefficients
print('Estimated intercept: ', regr.named_steps['ridge'].intercept_)

# The coefficients
print('Coefficients: ', regr.named_steps['ridge'].coef_)

# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(songs_y_test, songs_y_pred))